In [147]:
import pandas as pd
import numpy as np

data = pd.read_csv('IFR_Extract_with_selected_columns_15-5-23.csv')

obreak_date = pd.to_datetime(data.obreak_date)
datebone = pd.to_datetime(data.datebone)
y = ( abs( datebone - obreak_date))
X = data.drop(["obreak_date","datebone"],axis=1)
selectedColumns = [ 'PatientAge', "PatientGender",'parentbreak', 'alcohol',
                'arthritis', 'diabetes',
                'oralster', 'smoke', 'obreak']



In [148]:
dropList = []
for i in data:
    if data[i].dtypes == 'O':
        dropList.append(data[i].name)
dropList.append("CompletedSurveyId")
dropList.append("PatientId")
X = data.drop(dropList,axis=1)
X.fillna(0,inplace=True)
y = pd.DataFrame({"time":y})



In [149]:
y.time

y['event'] = y.time.apply(lambda x: x.days != 0 )
structured_array = y.to_records(index=False)

swapped = pd.DataFrame({
    "event": y.event,
    "time": y.time.apply(lambda x: x.days)
})
(swapped.event).value_counts()

True     792
False      8
Name: event, dtype: int64

In [150]:
swapped.event = swapped.event.astype(bool)
swapped.event
structured_array = np.rec.array(swapped.to_records(index=False))

In [151]:
swapped

,event,time
0,True,524
1,True,2046
2,True,15455
3,True,4354
4,True,2207
...,...,...
795,True,579
796,True,5109
797,True,2125
798,True,518


In [152]:
mergedBeforeEncoding = pd.concat([X[selectedColumns],swapped],axis=1)
mergedBeforeEncoding

,PatientAge,PatientGender,parentbreak,alcohol,arthritis,diabetes,oralster,smoke,obreak,event,time
0,53,1,0,0,0.0,0.0,0,0,1,True,524
1,85,1,0,0,0.0,1.0,0,0,1,True,2046
2,90,1,0,0,1.0,0.0,0,0,1,True,15455
3,81,1,0,0,0.0,0.0,0,0,1,True,4354
4,60,1,1,0,0.0,1.0,0,0,1,True,2207
...,...,...,...,...,...,...,...,...,...,...,...
795,83,1,4,0,0.0,0.0,4,0,1,True,579
796,60,1,0,0,0.0,0.0,0,0,1,True,5109
797,76,2,0,0,0.0,0.0,0,0,1,True,2125
798,61,1,2,1,0.0,0.0,0,1,1,True,518


In [153]:
PatientAge = []
PatientGender = []
parentbreak = []
alcohol = []
arthritis = []
diabetes = []
oralster = []
smoke = []
obreak = []
event = []
time = []
for i in range(400):
    PatientAge.append(np.random.randint(40,104))
    PatientGender.append( np.random.randint(1,3))
    parentbreak.append(np.random.randint(0,5))
    alcohol.append(np.random.randint(0,5))
    arthritis.append(np.random.randint(0,2))
    diabetes.append(np.random.randint(0,2))
    oralster.append(np.random.randint(0,5))
    smoke.append(np.random.randint(0,5))
    obreak.append(1)
    event.append(False)
    time.append(0)

synthetic_data = pd.DataFrame({
    "PatientAge":PatientAge,
    "PatientGender":PatientGender,
    "parentbreak":parentbreak,
    "alcohol":alcohol,
    "arthritis":arthritis,
    "diabetes":diabetes,
    "oralster":oralster,
    "smoke":smoke,
    "obreak":obreak,
    "event":event,
    "time":time
})
synthetic_data['arthritis'] = synthetic_data['arthritis'].astype(float)
synthetic_data['diabetes'] = synthetic_data['diabetes'].astype(float)



In [154]:
merged_synthetic = pd.concat([mergedBeforeEncoding,synthetic_data],axis=0)

merged_synthetic

,PatientAge,PatientGender,parentbreak,alcohol,arthritis,diabetes,oralster,smoke,obreak,event,time
0,53,1,0,0,0.0,0.0,0,0,1,True,524
1,85,1,0,0,0.0,1.0,0,0,1,True,2046
2,90,1,0,0,1.0,0.0,0,0,1,True,15455
3,81,1,0,0,0.0,0.0,0,0,1,True,4354
4,60,1,1,0,0.0,1.0,0,0,1,True,2207
...,...,...,...,...,...,...,...,...,...,...,...
395,77,1,4,4,0.0,1.0,0,4,1,False,0
396,59,2,4,2,0.0,1.0,0,2,1,False,0
397,64,2,1,0,1.0,1.0,0,3,1,False,0
398,45,1,2,1,1.0,0.0,3,1,1,False,0


In [146]:
cat_features = ['parentbreak', 'alcohol',
                'arthritis', 'diabetes',
                'oralster', 'smoke', 'obreak',
                # These features were determined to apply minimal impact even
                # 'respdisease', 'hbp','heartdisease',
                # 'ptunsteady', 'wasfractdue2fall', 'cholesterol',
                # 'ptfall', 'shoulder', 'wrist', 'bmdtest_10yr_caroc'
                ]
for feature in cat_features:
    if merged_synthetic is not None:
        cat_one_hot = pd.get_dummies(merged_synthetic[feature], prefix=f'{feature}', drop_first=False)
        data = data.drop(feature, axis=1)
        data = data.join(cat_one_hot)

KeyError: "['parentbreak'] not found in axis"

In [122]:
selectedColumns = [ 'PatientAge', "PatientGender",
                                 'parentbreak_1', 'alcohol_1', 'arthritis_1.0', 'diabetes_1.0',
                                 'oralster_1', 'oralster_2', 'smoke_1']
X = data[selectedColumns]


In [124]:
data

,CompletedSurveyId,PatientId,PatientGender,PatientAge,PostalCode,DateSurveyed,screeningmode,disclaimer_new,howbreak,eligible_hidden,...,oralster_1,oralster_2,oralster_3,oralster_4,smoke_0,smoke_1,smoke_2,smoke_3,smoke_4,obreak_1
0,179173,431192,1,53,H0H,5-Jul-21,1,1,1,1,...,0,0,0,0,1,0,0,0,0,1
0,179173,431192,1,53,H0H,5-Jul-21,1,1,1,1,...,0,0,0,0,1,0,0,0,0,1
0,179173,431192,1,53,H0H,5-Jul-21,1,1,1,1,...,0,0,0,0,0,1,0,0,0,1
0,179173,431192,1,53,H0H,5-Jul-21,1,1,1,1,...,0,0,0,0,0,1,0,0,0,1
0,179173,431192,1,53,H0H,5-Jul-21,1,1,1,1,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,204380,589168,1,83,H0H,15-Nov-22,1,1,1,1,...,0,0,0,1,1,0,0,0,0,1
796,204407,589179,1,60,H0H,16-Nov-22,1,1,1,1,...,0,0,0,0,1,0,0,0,0,1
797,204416,589182,2,76,H0H,16-Nov-22,1,1,1,1,...,0,0,0,0,1,0,0,0,0,1
798,204438,589191,1,61,H0H,16-Nov-22,1,1,1,1,...,0,0,0,0,0,1,0,0,0,1


55
81
55
58
82
72
78
83
60
68
69
85
67
73
63
68
82
88
82
79
76
72
63
86
79
84
82
80
72
52
82
74
69
57
63
66
73
66
74
67
67
55
66
78
72
79
83
77
85
56
52
53
80
67
57
83
63
85
67
55
89
63
55
52
88
63
78
89
63
81
63
67
69
50
57
51
83
66
64
73
69
68
73
69
78
59
59
69
82
55
85
88
63
59
89
71
88
62
76
59
59
86
77
61
80
54
66
82
80
54
82
75
84
59
89
74
57
68
63
69
68
87
80
70
61
77
79
62
53
56
70
82
79
68
66
80
50
82
50
77
68
73
72
81
84
82
84
57
75
68
59
66
63
68
57
83
85
89
85
86
63
50
57
87
74
80
86
75
61
83
51
78
60
56
73
50
52
80
70
75
50
66
72
51
52
60
84
87
74
74
89
63
60
61
55
61
89
58
70
60
86
51
56
64
